# GPU Polars vs cuDF Pandas

In the following notebook, we will compare between Polars and Pandas performance on GPU in various tasks.

## Install Requirements

In [4]:
!pip install faker
!pip install polars
!pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com

## Verify Hardware

In [6]:
!nvidia-smi

Thu Oct 24 03:11:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              32W /  70W |   7691MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load Data

In the following cell, we will generate a Polars Lazy Frame and a Pandas Data Frame using a dictionary with 10,000 random entries.
We will also compare between both structures in the end of the process to ensure they store the same data.

In [7]:
%load_ext cudf.pandas
import pandas as pd
import polars as pl
from faker import Faker
import random
import time

fake = Faker()

data = {
    "name": [fake.name() for i in range(10000)],
    "address": [fake.address() for i in range(10000)],
    "profession": [random.choice(["Chef", "Doctor", "Developer"]) for i in range(10000)],
    "salary": [random.randint(1, 1000000) for i in range(10000)]
}

pl_data = pl.LazyFrame(data)
pd_data = pd.DataFrame(data)

print(pl_data.head(3).collect(engine="gpu"))
print("####################################################################")
print(pd_data.head(3))

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas
shape: (3, 4)
┌───────────────┬──────────────────────────────┬────────────┬────────┐
│ name          ┆ address                      ┆ profession ┆ salary │
│ ---           ┆ ---                          ┆ ---        ┆ ---    │
│ str           ┆ str                          ┆ str        ┆ i64    │
╞═══════════════╪══════════════════════════════╪════════════╪════════╡
│ Cynthia Chang ┆ 23993 James Station Apt. 093 ┆ Developer  ┆ 91263  │
│               ┆ S…                           ┆            ┆        │
│ Jessica Cox   ┆ 798 Stanley Flat Apt. 619    ┆ Doctor     ┆ 576566 │
│               ┆ Cour…                        ┆            ┆        │
│ Javier King   ┆ 9544 Norman Corner           ┆ Chef       ┆ 361178 │
│               ┆ Reeseboroug…                 ┆            ┆        │
└───────────────┴──────────────────────────────┴────────────┴────────┘
############################################

## Group By Statement

We will perform the same operation across Polars and Pandas, and compair the results.

### GPU Polars

In [8]:
for i in range(3):
    print("run", i)
    print("----------------------------")
    %time pl_data.group_by("profession").agg(pl.min("salary").alias("min"),pl.max("salary").alias("max"),pl.mean("salary").alias("avg"),pl.count("salary").alias("count")).collect(engine="gpu")
    print("----------------------------")

%time pl_data.group_by("profession").agg(pl.min("salary").alias("min"),pl.max("salary").alias("max"),pl.mean("salary").alias("avg"),pl.count("salary").alias("count")).collect(engine="gpu")

run 0
----------------------------
CPU times: user 6.89 ms, sys: 893 µs, total: 7.79 ms
Wall time: 6.92 ms
----------------------------
run 1
----------------------------
CPU times: user 4.85 ms, sys: 0 ns, total: 4.85 ms
Wall time: 4.68 ms
----------------------------
run 2
----------------------------
CPU times: user 4.59 ms, sys: 0 ns, total: 4.59 ms
Wall time: 4.39 ms
----------------------------
CPU times: user 3.1 ms, sys: 849 µs, total: 3.95 ms
Wall time: 3.73 ms


profession,min,max,avg,count
str,i64,i64,f64,u32
"""Doctor""",77,999970,509942.660856,3270
"""Developer""",465,999980,500198.771378,3368
"""Chef""",901,999728,499816.211481,3362


### cuDF Pandas

In [9]:
for i in range(3):
  print("run", i)
  print("----------------------------")
  %time pd_data.groupby("profession").agg({'salary': ['min', 'max', 'mean', 'count']})
  print("----------------------------")

%time pd_data.groupby("profession").agg({'salary': ['min', 'max', 'mean', 'count']})

run 0
----------------------------
CPU times: user 6.78 ms, sys: 1.98 ms, total: 8.77 ms
Wall time: 11.4 ms
----------------------------
run 1
----------------------------
CPU times: user 5.84 ms, sys: 2.89 ms, total: 8.73 ms
Wall time: 8.15 ms
----------------------------
run 2
----------------------------
CPU times: user 12.6 ms, sys: 1.96 ms, total: 14.6 ms
Wall time: 14.1 ms
----------------------------
CPU times: user 7.18 ms, sys: 918 µs, total: 8.1 ms
Wall time: 6.99 ms


salary                             
              min     max           mean count
profession                                    
Chef          901  999728  499816.211481  3362
Developer     465  999980  500198.771378  3368
Doctor         77  999970  509942.660856  3270